In [2]:
import argparse
import os
import numpy as np
import math
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs('images', exist_ok=True)
os.makedirs('saved_models', exist_ok=True)

channels = 1
img_size = 28
n_classes = 10
latent_dim = 100
batch_size = 32
learning_rate = .0002
b1 = .5
b2 = .999
sample_interval = 1

img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [  nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim+n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(n_classes, n_classes)
        
        self.model = nn.Sequential(
            nn.Linear(n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1)
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity
    
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

# Loss functions
adversarial_loss = torch.nn.MSELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

In [3]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row**2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, 'images/%d.png' % batches_done, nrow=n_row, normalize=True)

In [47]:
class QuickDrawDataset(Dataset):
    """Quick Draw dataset."""

    def __init__(self, label, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = np.apply_along_axis(self.__reshape_row, 1, np.load(f'../data/{label}.npy'))
        
        self.label = label
        self.transform = transform
    
    def __reshape_row(self, row):
        return np.reshape(row, img_shape)

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        image = self.data_frame[idx]
        label = self.label
        return image, 1

In [48]:
data = QuickDrawDataset(label='panda', transform=transforms.Compose([
                        transforms.Resize(img_size),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
dataloader = DataLoader(data, batch_size=32, shuffle=False)

In [53]:
training_gen_loss = []
training_dis_loss = []

for epoch in range(1):
    mean_dis_loss = []
    mean_gen_loss = []
    for i, (imgs, labels) in enumerate(dataloader):
        batch_size = len(imgs)
        
        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)
        
        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor)) # 32x794
        labels = Variable(labels.type(LongTensor))
        print(real_imgs.shape)
        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, 1, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        mean_dis_loss.append(d_loss.item())
        mean_gen_loss.append(g_loss.item())
        
        print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" % (epoch, 1, i, len(dataloader),
                                                            d_loss.item(), g_loss.item()))
        
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)
        if i > 1: break
    training_gen_loss.append(np.mean(mean_dis_loss))
    training_dis_loss.append(np.mean(mean_gen_loss))

torch.Size([32, 1, 28, 28])
[Epoch 0/1] [Batch 0/3551] [D loss: 6.404902] [G loss: 0.936876]
torch.Size([32, 1, 28, 28])
[Epoch 0/1] [Batch 1/3551] [D loss: 7.777600] [G loss: 0.930169]
torch.Size([32, 1, 28, 28])
[Epoch 0/1] [Batch 2/3551] [D loss: 6.561669] [G loss: 0.924610]


In [54]:
def save_model():
    torch.save({
            'epoch': epoch,
            'generator': generator.model.state_dict(),
            'discriminator': discriminator.model.state_dict(),
            'optimizer_gen': optimizer_G.state_dict(),
            'optimizer_dis': optimizer_D.state_dict(),
            'G_loss': training_gen_loss,
            'D_loss': training_dis_loss
            }, 'saved_models/models')

In [30]:
def load_model(model, path):
    check_point = torch.load('saved_models/models', map_location='cpu')
    m.load_state_dict(torch.load(path), strict=False)
    return m.eval()

In [57]:
torch.load('../models', map_location='cpu')

{'optimizer_gen': {'state': {139824927127232: {'exp_avg_sq': tensor([[1.1305e-10, 2.4519e-11, 4.0035e-11,  ..., 1.5557e-10, 1.1345e-10,
             1.4050e-10],
            [1.4696e-11, 3.1620e-12, 5.1877e-12,  ..., 3.6955e-11, 3.3537e-11,
             3.4745e-11],
            [4.3371e-10, 9.4234e-11, 1.5377e-10,  ..., 4.8635e-10, 4.3522e-10,
             4.6956e-10],
            ...,
            [5.0394e-10, 1.1409e-10, 1.8232e-10,  ..., 1.3294e-09, 1.1695e-09,
             1.2208e-09],
            [6.3761e-10, 1.3650e-10, 2.2431e-10,  ..., 8.4581e-10, 7.0134e-10,
             8.4311e-10],
            [3.7737e-10, 8.0920e-11, 1.3292e-10,  ..., 4.9097e-10, 4.6045e-10,
             5.4691e-10]]),
    'step': 35510,
    'exp_avg': tensor([[-2.5081e-06, -1.2069e-06,  1.5176e-06,  ...,  1.6179e-06,
             -3.9604e-07,  8.5775e-08],
            [-1.5619e-07, -7.5162e-08,  9.4509e-08,  ...,  4.1387e-07,
             -1.3108e-07, -7.9314e-08],
            [-4.6366e-06, -2.2313e-06,  2.